#  GAN

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
* https://www.kaggle.com/datasets/splcher/animefacedataset
* https://github.com/eriklindernoren/PyTorch-GAN

## Задачи для совместного разбора

1\. Обсудите основные шаги в обучении GAN.

In [3]:
import torch.nn as nn
import torch

gen = nn.Sequential(
    nn.ConvTranspose2d(100, 8, 4, 2, 1),
    nn.Tanh()
)

In [4]:
noise = torch.randn(16, 100, 1, 1)

In [6]:
generated = gen(noise)
generated.shape

torch.Size([16, 8, 2, 2])

In [7]:
faked_labels = torch.zeros(16)

In [10]:
real = torch.randn(16, 8, 2, 2)
real_labels = torch.ones(16)

In [14]:
discriminator = nn.Sequential(
    #...
    nn.Flatten(start_dim=1),
    nn.LazyLinear(out_features=2)
)

In [15]:
discriminator(generated).shape

torch.Size([16, 2])

In [16]:
discriminator(real).shape

torch.Size([16, 2])

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Создайте набор данных на основе датасета `animefacedataset`. Используя преобразования `torchvision`, приведите изображения к одному размеру и нормализуйте их. Выведите на экран несколько примеров изображений.

- [ ] Проверено на семинаре

<p class="task" id="2"></p>

2\. Реализуйте архитектуру `DCGAN` и обучите модель. Подберите гиперпараметры таким образом, чтобы получаемые изображения стали достаточного качественными (четкими и без существенных дефектов). Во время обучения сохраняйте примеры генерации изображений из случайного шума и сравните, как менялось качество получаемых изображений в процессе обучения.

- [ ] Проверено на семинаре

<p class="task" id="3"></p>

3\. Создайте наборы данных на основе архива `summer2winter_yosemite.zip`. Используя преобразования `torchvision`, приведите изображения к одному размеру и нормализуйте их. Выведите на экран несколько примеров изображений, расположив изображения из одной пары рядом по горизонтали.

- [ ] Проверено на семинаре

<p class="task" id="4"></p>

4\. Реализуйте архитектуру `CycleGAN` и обучите модель. Подберите гиперпараметры таким образом, чтобы получаемые изображения стали достаточного качественными (четкими и без существенных дефектов). Во время обучения сохраняйте примеры преобразования (в обе стороны) и  сравните, как менялось качество получаемых изображений в процессе обучения.

- [ ] Проверено на семинаре